In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
fundamentals = pd.read_csv('fundamenals.csv')
dividendsIDK = pd.read_csv('Dividend.csv')
#merged_df = pd.read_csv('merged_df.csv')

In [ ]:
fundamentals

In [ ]:
dividendsIDK

In [ ]:
fundamentals = fundamentals.rename(columns={
    'tic': 'ticker',

})

In [ ]:
fundamentals

In [ ]:
df = pd.merge(dividendsIDK, fundamentals, on='ticker')
df

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
print("Missing values distribution: ")
print(df.isnull().mean())
print("")

In [ ]:
'''drop ones with high percentage%'''

In [ ]:
df = df.drop(columns=['capgn','cheqv','div','divd','divsp','anncdate','cheqvpaydate','divdpaydate','adrrc'])

In [ ]:
print("Missing values distribution: ")
print(df.isnull().mean())
print("")

In [ ]:
df = df.drop(columns=['spcsrc'])

In [ ]:
print("Column datatypes: ")
print(df.dtypes)

In [ ]:
print(df.duplicated().sum())

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.dropna()

In [ ]:
print("Missing values distribution: ")
print(df.isnull().mean())
print("")

In [ ]:
df.dropna(subset=['cshoc', 'cshtrd', 'prcod', 'trfd', 'ggroup', 'gind', 'gsector', 'gsubind'], inplace=True)
df

In [ ]:
print("Missing values distribution: ")
print(df.isnull().mean())
print("")

In [ ]:
df = df.dropna()
df

In [ ]:
# further clean
columns_to_delete = ['nrets_est', 'gvkey', 'iid', 'datadate']
df = df.drop(columns=columns_to_delete)

In [ ]:
df.head()

In [ ]:
import pandas as pd

# Convert the "evtdate" column to a datetime type if it's not already
df['evtdate'] = pd.to_datetime(df['evtdate'])

# Subset the DataFrame to select data after 2010-01-01
#subset_df = df[df['evtdate'] > '2020-01-01']

subset_df = df[df['gsector'] == 45]

In [ ]:
import pandas as pd

# Read the Div_new.csv file into a DataFrame
div_df = pd.read_csv("Div_new.csv")

columns_to_delete_new = ['uid']
div_df = div_df.drop(columns=columns_to_delete_new)

# Merge the datasets based on the "ticker" column
merged_df = pd.merge(subset_df, div_df, on="ticker")

# View the merged DataFrame
print(merged_df)

merged_df['marketcap'] = merged_df['prccd'] * merged_df['cshoc']

#df.to_csv('merged_df.csv', index=False)

# BEFORE EVENT HAPPENS

### Predict ABRET using fundamentals before/after the event happens

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# assuming the dataframe is named merged_df

# Define the predictors
predictors = ['dvi', 'eps', 'prccd', 'prchd', 'prcld', 'trfd', 'ggroup', 'gind', 'gsubind', 'cshtrd', 'marketcap']

# Initialize a dictionary to hold the results
regression_results = {}

# Iterate over the event times
for t in range(-10, 0):
    # Select the rows for the given event time
    df_t = merged_df[merged_df['evttime'] == t]
    
    # Form the regression formula
    formula = 'abret ~ ' + ' + '.join(predictors)
    
    # Run the regression
    result = smf.ols(formula, data=df_t).fit()
    
    # Store the results
    regression_results[t] = result

# Print the results
for t, result in regression_results.items():
    print(f'Event Time {t}:')
    print(result.summary())
    print('\n\n')


### predcit car (same technique)

In [ ]:
import pandas as pd

# Read the Div_new.csv file into a DataFrame
div_df = pd.read_csv("Div_new.csv")

columns_to_delete_new = ['uid']
div_df = div_df.drop(columns=columns_to_delete_new)

# Merge the datasets based on the "ticker" column
merged_df_car = pd.merge(subset_df, div_df, on="ticker")

In [ ]:
merged_df_car['marketcap'] = merged_df_car['prccd'] * merged_df_car['cshoc']

# Calculate cumulative abnormal return
merged_df_car = merged_df_car.sort_values(by=['ticker', 'evtdate_x', 'evttime'])

# Group by Ticker and evtdate, then calculate the rolling sum of 'abret' from evttime -10 to evttime -1
merged_df_car['car'] = merged_df_car.groupby(['ticker', 'evtdate_x'])['abret'].transform(lambda x: x.rolling(10, min_periods=1).sum())

# Select rows where evttime is -1 (the end of the window)
car_df = merged_df_car[merged_df_car['evttime'] == -1]

# Define the predictors
predictors = ['dvi', 'eps', 'prccd', 'prchd', 'prcld', 'trfd', 'ggroup', 'gind', 'gsubind', 'cshtrd', 'marketcap']

# Initialize a dictionary to hold the results
car_regression_results = {}

# Iterate over the event times
for t in range(-10, 0):
    # Select the rows for the given event time
    df_t = merged_df_car[merged_df_car['evttime'] == t]
    
    # Form the regression formula
    formula = 'car ~ ' + ' + '.join(predictors)
    
    # Run the regression
    result = smf.ols(formula, data=df_t).fit()
    
    # Store the results
    car_regression_results[t] = result

# Print the results
for t, result in car_regression_results.items():
    print(f'Event Time {t}:')
    print(result.summary())
    print('\n\n')

# AFTER EVENT HAPPENS

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

merged_df['marketcap'] = merged_df['prccd'] * merged_df['cshoc']

# Define the predictors
predictors = ['dvi', 'eps', 'prccd', 'prchd', 'prcld', 'trfd', 'ggroup', 'gind', 'gsubind', 'cshtrd', 'marketcap']

# Initialize a dictionary to hold the results
afterregression_results = {}

# Iterate over the event times
for t in range(0, 10):
    # Select the rows for the given event time
    df_t = merged_df[merged_df['evttime'] == t]
    
    # Form the regression formula
    formula = 'abret ~ ' + ' + '.join(predictors)
    
    # Run the regression
    result = smf.ols(formula, data=df_t).fit()
    
    # Store the results
    afterregression_results[t] = result

# Print the results
for t, result in afterregression_results.items():
    print(f'Event Time {t}:')
    print(result.summary())
    print('\n\n')


In [ ]:
# Read the Div_new.csv file into a DataFrame
div_df = pd.read_csv("Div_new.csv")

columns_to_delete_new = ['uid']
div_df = div_df.drop(columns=columns_to_delete_new)

# Merge the datasets based on the "ticker" column
merged_df_car = pd.merge(subset_df, div_df, on="ticker")
merged_df_car['marketcap'] = merged_df_car['prccd'] * merged_df_car['cshoc']


# Calculate cumulative abnormal return
merged_df_car = merged_df.sort_values(by=['ticker', 'evtdate_x', 'evttime'])

# Group by Ticker and evtdate, then calculate the rolling sum of 'abret' from evttime -10 to evttime -1
merged_df_car['car'] = merged_df_car.groupby(['ticker', 'evtdate_x'])['abret'].transform(lambda x: x.rolling(10, min_periods=1).sum())

# Select rows where evttime is -1 (the end of the window)
car_df = merged_df_car[merged_df_car['evttime'] == -1]

# Define the predictors
predictors = ['dvi', 'eps', 'prccd', 'prchd', 'prcld', 'trfd', 'ggroup', 'gind', 'gsubind', 'cshtrd', 'marketcap']

# Initialize a dictionary to hold the results
aftercar_regression_results = {}

# Iterate over the event times
for t in range(0, 10):
    # Select the rows for the given event time
    df_t = merged_df_car[merged_df_car['evttime'] == t]
    
    # Form the regression formula
    formula = 'car ~ ' + ' + '.join(predictors)
    
    # Run the regression
    result = smf.ols(formula, data=df_t).fit()
    
    # Store the results
    aftercar_regression_results[t] = result

# Print the results
for t, result in aftercar_regression_results.items():
    print(f'Event Time {t}:')
    print(result.summary())
    print('\n\n')

# test the significances of the variables above

### best day to invest in Company after/before the event happens

# compare the outcomes when inputs are the same
# rank them

### compare the results and conclude
1. if knowing the event_div_cut will happen (long time ago, as long as 10 days), what is the best day to invest?
2. after the event_div_cut happened (as long as 10 days), what is the best day to invest?

# find the best day

### Pre- EVENT

In [ ]:
# Extract intercepts
abret_intercepts = {t: result.params['Intercept'] for t, result in regression_results.items()}
car_intercepts = {t: result.params['Intercept'] for t, result in car_regression_results.items()}

# Sort by intercept
abret_sorted = sorted(abret_intercepts.items(), key=lambda item: item[1])
car_sorted = sorted(car_intercepts.items(), key=lambda item: item[1])

# Print the sorted results
print("Ranking based on the value of the intercept for 'abret':")
for t, intercept in abret_sorted:
    print(f'Event Time {t}: Intercept = {intercept}')
    
print("\nRanking based on the value of the intercept for 'car':")
for t, intercept in car_sorted:
    print(f'Event Time {t}: Intercept = {intercept}')


### After -EVENT

In [ ]:
# Extract intercepts
abret_intercepts = {t: result.params['Intercept'] for t, result in afterregression_results.items()}
car_intercepts = {t: result.params['Intercept'] for t, result in aftercar_regression_results.items()}

# Sort by intercept
abret_sorted = sorted(abret_intercepts.items(), key=lambda item: item[1])
car_sorted = sorted(car_intercepts.items(), key=lambda item: item[1])

# Print the sorted results
print("Ranking based on the value of the intercept for 'abret':")
for t, intercept in abret_sorted:
    print(f'Event Time {t}: Intercept = {intercept}')
    
print("\nRanking based on the value of the intercept for 'car':")
for t, intercept in car_sorted:
    print(f'Event Time {t}: Intercept = {intercept}')


# Sentimental Analysis using
sec edgar parsing python  AND
sec-api · PyPI
